# 🧠 Using MOISE+MARL in Warehouse Management

This notebook demonstrates how to apply the **MOISE+MARL framework** to a **warehouse logistics environment**, where agents coordinate to transport objects and fulfill delivery roles.

## 1. Import Dependencies

In [ ]:
import math
import gym
import random
import numpy as np
from marllib import marl
from random import randint
from mma_wrapper.label_manager import label_manager
from mma_wrapper.organizational_model import (
    organizational_model, structural_specifications,
    functional_specifications, deontic_specifications,
    deontic_specification, time_constraint_type
)
from mma_wrapper.organizational_specification_logic import role_logic
from mma_wrapper.utils import label, observation, action, trajectory
from marllib.envs.base_env.wmt import RLlibWMT

## 2. Define the Label Manager

In [ ]:
class wm_label_manager(label_manager):
    def __init__(self, action_space=None, observation_space=None, view_scope=None):
        super().__init__(action_space, observation_space)
        self.view_scope = view_scope
        self.action_encode = {"nothing": 0, "up": 1, "down": 2, "left": 3, "right": 4, "pick": 5, "drop": 6}
        self.action_decode = {v: k for k, v in self.action_encode.items()}
        self.cell_encode = {
            "empty": 1, "obstacle": 0, "agent": 2, "agent_with_primary": 3,
            "agent_with_secondary": 4, "primary_object": 5, "secondary_object": 6,
            "empty_input": 7, "input_with_object": 8, "empty_input_craft": 9,
            "input_craft_with_object": 10, "empty_output_craft": 11,
            "output_craft_with_object": 12, "empty_output": 13, "output_with_object": 14
        }
        self.cell_decode = {v: k for k, v in self.cell_encode.items()}

    def one_hot_encode_observation(self, observation, agent=None):
        obs = np.asarray(observation).reshape(-1)
        return np.asarray([self.cell_encode[cell] for cell in obs])

    def one_hot_decode_observation(self, observation, agent=None):
        obs = np.asarray([self.cell_decode[val] for val in observation])
        if self.view_scope:
            return obs.reshape((self.view_scope * 2 + 1), (self.view_scope * 2 + 1))
        return obs

    def one_hot_encode_action(self, action, agent=None):
        return self.action_encode[action]

    def one_hot_decode_action(self, action, agent=None):
        return self.action_decode[action]

## 3. Define Role Logic Functions

In [ ]:
def primary_fun(trajectory, observation, agent_name, label_manager):
    data = label_manager.one_hot_decode_observation(observation, agent=agent_name)
    x, y = data.shape[0] // 2, data.shape[1] // 2
    if randint(0, 100) < 20:
        return randint(1, 4)
    if data[x, y] == "agent_with_primary":
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                if data[i, j] == "empty_input_craft":
                    dx, dy = i - x, j - y
                    if abs(dx) == 0 and abs(dy) == 1:
                        return 6
                    return [1, 2][dx > 0] if abs(dx) >= abs(dy) else [3, 4][dy > 0]
        return 3
    if data[x, y] == "agent":
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                if data[i, j] == "input_with_object":
                    dx, dy = i - x, j - y
                    if abs(dx) == 0 and abs(dy) == 1:
                        return 5
                    return [1, 2][dx > 0] if abs(dx) >= abs(dy) else [3, 4][dy > 0]
        return 3
    return 0

def secondary_fun(trajectory, observation, agent_name, label_manager):
    data = label_manager.one_hot_decode_observation(observation, agent=agent_name)
    def block_around(x, y, grid, block):
        for dx, dy in [(1, 0), (0, 1), (0, -1), (-1, 0)]:
            if grid[x+dx][y+dy] == block:
                return True
    if randint(0, 100) < 40:
        return randint(1, 4)
    x, y = data.shape[0] // 2, data.shape[1] // 2
    if data[x, y] == "agent_with_secondary":
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                if data[i, j] == "empty_output":
                    dx, dy = i - x, j - y
                    if abs(dx) == 0 and abs(dy) == 1:
                        return 6
                    return [1, 2][dx > 0] if abs(dx) >= abs(dy) else [3, 4][dy > 0]
        return 4
    if data[x, y] == "agent":
        if block_around(x, y, data, "output_craft_with_object"):
            return 5
        for i in range(data.shape[0]):
            for j in range(data.shape[1]):
                if data[i, j] == "output_craft_with_object":
                    dx, dy = i - x, j - y
                    return [1, 2][dx > 0] if abs(dx) >= abs(dy) else [3, 4][dy > 0]
        return random.choice([2, 4]) if randint(0, 100) < 70 else randint(1, 4)
    return 0

## 4. Setup Environment and Organizational Model

In [ ]:
_env = RLlibWMT({"map_name": "warehouse_management"})
view_scope = _env.env.par_env.view_size
wm_label_mngr = wm_label_manager(view_scope=view_scope)

wm_model = organizational_model(
    structural_specifications(
        roles={
            "role_primary": role_logic(label_manager=wm_label_mngr).registrer_script_rule(primary_fun),
            "role_secondary": role_logic(label_manager=wm_label_mngr).registrer_script_rule(secondary_fun)
        },
        role_inheritance_relations={}, root_groups={}
    ),
    functional_specifications=functional_specifications(goals={}, social_scheme={}, mission_preferences=[]),
    deontic_specifications=deontic_specifications(
        obligations=[
            deontic_specification("role_primary", ["agent_0", "agent_1"], [], time_constraint_type.ANY),
            deontic_specification("role_secondary", ["agent_2"], [], time_constraint_type.ANY)
        ],
        permissions=[]
    )
)

## 5. Create Environment with MOISE+MARL

In [ ]:
env = marl.make_env(
    environment_name="wmt",
    map_name="warehouse_management",
    organizational_model=wm_model
)

## 6. Train or Render the Environment

In [ ]:
mappo = marl.algos.mappo(hyperparam_source="test")
model = marl.build_model(env, mappo, {"core_arch": "mlp", "encode_layer": "128-256"})

# mappo.fit(env, model, stop={'timesteps_total': 2e6})  # Optional training

mappo.render(env, model,
    restore_path={
        'params_path': "./exp_results/mappo_mlp_warehouse_management_copy/.../params.json",
        'model_path': "./exp_results/mappo_mlp_warehouse_management_copy/.../checkpoint-20",
        'render_env': True
    },
    local_mode=True,
    share_policy="group"
)

## ✅ Conclusion
In this notebook:
- We defined agent roles for warehouse logistics
- Built a MOISE+ organizational model with script-based role logic
- Wrapped the environment and rendered agent behavior accordingly